In [1]:
import pandas as pd
import numpy as np
import spacy
nlp=spacy.load("en")

In [2]:
import re
def clearstring(string):
    string = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', string, flags=re.MULTILINE)
    string = re.sub('[^@#A-Za-z0-9 ]+', '', string)
    string = string.split(' ')
    string = filter(None, string)
    string = [y.strip() for y in string]
    string = ' '.join(string)
    return string.lower()

In [3]:
df = pd.read_csv('gender-classifier-DFE-791531.csv', encoding = "ISO-8859-1")
df.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN


In [17]:
matrix_description = df[['text', 'description', 'gender']].values
for i in range(matrix_description.shape[0]):
    try:
        matrix_description[i, 0] = clearstring(matrix_description[i, 0])
    except:
        matrix_description[i, 0] = ''
    try:
        matrix_description[i, 1] = clearstring(matrix_description[i, 1])
    except:
        matrix_description[i, 1] = ''

In [19]:
import pickle
vocab = (' '.join(matrix_description[:, 0].tolist())).split()
vocab += (' '.join(matrix_description[:, 1].tolist())).split()
unique_vocab = list(set(vocab))
with open('vocab.p', 'wb') as fopen:
    pickle.dump(unique_vocab, fopen)

In [6]:
def build_dictionary(words, vocabulary_size):
    import collections
    count = []
    count.extend(collections.Counter(words).most_common(vocabulary_size))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = []
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        data.append(index)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, dictionary, reverse_dictionary

In [20]:
_, dictionary, reverse_dictionary = build_dictionary(vocab, len(unique_vocab))

In [21]:
with open('dictionary.p', 'wb') as fopen:
    pickle.dump(dictionary, fopen)
with open('dict_reverse.p', 'wb') as fopen:
    pickle.dump(reverse_dictionary, fopen)

In [26]:
list_unique = np.unique(matrix_description[:, 2].astype('str')).tolist()
del list_unique[3]

['brand', 'female', 'male', 'nan', 'unknown']


In [31]:
for i in list_unique:
    lists = matrix_description[np.where(matrix_description[:, 2] == i)[0], :]
    with open(i, 'w') as fopen:
        for k in range(lists.shape[0] - 1):
            fopen.write(lists[k, 0] + '\n' + lists[k, 1] + '\n')
        fopen.write(lists[-1, 0] + '\n' + lists[-1, 1])

In [52]:
for i in list_unique:
    with open(i, 'r') as fopen:
        lists = fopen.read().split('\n')
    lists = list(filter(None, lists))
    with open(i, 'w') as fopen:
        for k in range(len(lists) - 1):
            fopen.write(lists[k] + '\n')
        fopen.write(lists[-1])
    lists_name_entity = []
    for k in range(len(lists)):
        tokens = nlp(lists[k])
        lists[k] = ' '.join([str(token) for token in tokens if token.pos_ == 'VERB' or token.pos_ == 'NOUN'])
        lists_name_entity += [str(token) for token in tokens if token.ent_type_ == 'ORG' or token.ent_type_ == 'PERSON' or token.ent_type_ == 'GPE']
    with open(i + '_pos', 'w') as fopen:
        for k in range(len(lists) - 1):
            fopen.write(lists[k] + '\n')
        fopen.write(lists[-1])
    lists = list(set((' '.join(lists)).split()))
    with open(i + '_token', 'w') as fopen:
        for k in range(len(lists) - 1):
            fopen.write(lists[k] + '\n')
        fopen.write(lists[-1])
    lists_name_entity = list(set(lists_name_entity))
    with open(i + '_entity', 'w') as fopen:
        for k in range(len(lists_name_entity) - 1):
            fopen.write(lists_name_entity[k] + '\n')
        fopen.write(lists_name_entity[-1])

In [53]:
import os
list_emotion = os.listdir('/home/huseinzol05/Desktop/tree/emotion/data')
print(list_emotion)

['sadness', 'anger', 'fear', 'joy', 'surprise', 'love']


In [55]:
for i in list_emotion:
    direct = '/home/huseinzol05/Desktop/tree/emotion/data/' + i + '/'
    with open(direct + os.listdir(direct)[0] , 'r') as fopen:
        lists = fopen.read().split('\n')
    lists = list(filter(None, lists))
    with open(i, 'w') as fopen:
        for k in range(len(lists) - 1):
            fopen.write(lists[k] + '\n')
        fopen.write(lists[-1])
    lists_name_entity = []
    for k in range(len(lists)):
        tokens = nlp(lists[k])
        lists[k] = ' '.join([str(token) for token in tokens if token.pos_ == 'VERB' or token.pos_ == 'NOUN'])
        lists_name_entity += [str(token) for token in tokens if token.ent_type_ == 'ORG' or token.ent_type_ == 'PERSON' or token.ent_type_ == 'GPE']
    with open(i + '_pos', 'w') as fopen:
        for k in range(len(lists) - 1):
            fopen.write(lists[k] + '\n')
        fopen.write(lists[-1])
    lists = list(set((' '.join(lists)).split()))
    with open(i + '_token', 'w') as fopen:
        for k in range(len(lists) - 1):
            fopen.write(lists[k] + '\n')
        fopen.write(lists[-1])
    lists_name_entity = list(set(lists_name_entity))
    with open(i + '_entity', 'w') as fopen:
        for k in range(len(lists_name_entity) - 1):
            fopen.write(lists_name_entity[k] + '\n')
        fopen.write(lists_name_entity[-1])

In [56]:
words = []
for i in list_emotion:
    direct = '/home/huseinzol05/Desktop/tree/emotion/data/' + i + '/'
    with open(direct + os.listdir(direct)[0] , 'r') as fopen:
        words += fopen.read().split('\n')

In [58]:
vocab = list(set((' '.join(words)).split()))
unique_vocab = list(set(vocab))
with open('vocab_emotion.p', 'wb') as fopen:
    pickle.dump(unique_vocab, fopen)

In [60]:
_, dictionary, reverse_dictionary = build_dictionary(vocab, len(unique_vocab))
with open('dictionary_emotion.p', 'wb') as fopen:
    pickle.dump(dictionary, fopen)
with open('dict_reverse_emotion.p', 'wb') as fopen:
    pickle.dump(reverse_dictionary, fopen)

In [63]:
list_irony = os.listdir('/home/huseinzol05/Desktop/tree/irony/data')
print(list_irony)

['negative', 'positive']


In [64]:
for i in list_irony:
    direct = '/home/huseinzol05/Desktop/tree/irony/data/' + i + '/'
    with open(direct + os.listdir(direct)[0] , 'r') as fopen:
        lists = fopen.read().split('\n')
    lists = list(filter(None, lists))
    with open(i, 'w') as fopen:
        for k in range(len(lists) - 1):
            lists[k] = clearstring(lists[k])
            fopen.write(lists[k] + '\n')
        lists[-1] = clearstring(lists[-1])
        fopen.write(lists[-1])
    lists_name_entity = []
    for k in range(len(lists)):
        tokens = nlp(lists[k])
        lists[k] = ' '.join([str(token) for token in tokens if token.pos_ == 'VERB' or token.pos_ == 'NOUN'])
        lists_name_entity += [str(token) for token in tokens if token.ent_type_ == 'ORG' or token.ent_type_ == 'PERSON' or token.ent_type_ == 'GPE']
    with open(i + '_pos', 'w') as fopen:
        for k in range(len(lists) - 1):
            fopen.write(lists[k] + '\n')
        fopen.write(lists[-1])
    lists = list(set((' '.join(lists)).split()))
    with open(i + '_token', 'w') as fopen:
        for k in range(len(lists) - 1):
            fopen.write(lists[k] + '\n')
        fopen.write(lists[-1])
    lists_name_entity = list(set(lists_name_entity))
    with open(i + '_entity', 'w') as fopen:
        for k in range(len(lists_name_entity) - 1):
            fopen.write(lists_name_entity[k] + '\n')
        fopen.write(lists_name_entity[-1])

In [70]:
words = []
for i in list_irony:
    direct = '/home/huseinzol05/Desktop/tree/irony/data/' + i + '/'
    with open(direct + os.listdir(direct)[0] , 'r') as fopen:
        words += fopen.read().split('\n')
        
vocab = list(set((' '.join(words)).split()))
unique_vocab = list(set(vocab))
with open('vocab_irony.p', 'wb') as fopen:
    pickle.dump(unique_vocab, fopen)
_, dictionary, reverse_dictionary = build_dictionary(vocab, len(unique_vocab))
with open('dictionary_irony.p', 'wb') as fopen:
    pickle.dump(dictionary, fopen)
with open('dict_reverse_irony.p', 'wb') as fopen:
    pickle.dump(reverse_dictionary, fopen)

In [69]:
list_polarity = os.listdir('/home/huseinzol05/Desktop/tree/polarity/data')
print(list_polarity)

['negative', 'positive']


In [71]:
for i in list_polarity:
    direct = '/home/huseinzol05/Desktop/tree/polarity/data/' + i + '/'
    with open(direct + os.listdir(direct)[0] , 'r') as fopen:
        lists = fopen.read().split('\n')
    lists = list(filter(None, lists))
    with open(i, 'w') as fopen:
        for k in range(len(lists) - 1):
            lists[k] = clearstring(lists[k])
            fopen.write(lists[k] + '\n')
        lists[-1] = clearstring(lists[-1])
        fopen.write(lists[-1])
    lists_name_entity = []
    for k in range(len(lists)):
        tokens = nlp(lists[k])
        lists[k] = ' '.join([str(token) for token in tokens if token.pos_ == 'VERB' or token.pos_ == 'NOUN'])
        lists_name_entity += [str(token) for token in tokens if token.ent_type_ == 'ORG' or token.ent_type_ == 'PERSON' or token.ent_type_ == 'GPE']
    with open(i + '_pos', 'w') as fopen:
        for k in range(len(lists) - 1):
            fopen.write(lists[k] + '\n')
        fopen.write(lists[-1])
    lists = list(set((' '.join(lists)).split()))
    with open(i + '_token', 'w') as fopen:
        for k in range(len(lists) - 1):
            fopen.write(lists[k] + '\n')
        fopen.write(lists[-1])
    lists_name_entity = list(set(lists_name_entity))
    with open(i + '_entity', 'w') as fopen:
        for k in range(len(lists_name_entity) - 1):
            fopen.write(lists_name_entity[k] + '\n')
        fopen.write(lists_name_entity[-1])

In [73]:
words = []
for i in list_polarity:
    direct = '/home/huseinzol05/Desktop/tree/polarity/data/' + i + '/'
    with open(direct + os.listdir(direct)[0] , 'r') as fopen:
        words += fopen.read().split('\n')
        
vocab = list(set((' '.join(words)).split()))
unique_vocab = list(set(vocab))
with open('vocab_polarity.p', 'wb') as fopen:
    pickle.dump(unique_vocab, fopen)
_, dictionary, reverse_dictionary = build_dictionary(vocab, len(unique_vocab))
with open('dictionary_polarity.p', 'wb') as fopen:
    pickle.dump(dictionary, fopen)
with open('dict_reverse_polarity.p', 'wb') as fopen:
    pickle.dump(reverse_dictionary, fopen)

In [76]:
df = pd.read_csv('/home/huseinzol05/Desktop/tree/politics/Political-media-DFE.csv', encoding = "ISO-8859-1")
df.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,audience,audience:confidence,bias,bias:confidence,message,...,orig__golden,audience_gold,bias_gold,bioid,embed,id,label,message_gold,source,text
0,766192484,False,finalized,1,8/4/15 21:17,national,1.0,partisan,1.0,policy,...,NaN,NaN,NaN,R000596,"<blockquote class=""twitter-tweet"" width=""450"">...",3.83249E+17,From: Trey Radel (Representative from Florida),NaN,twitter,RT @nowthisnews: Rep. Trey Radel (R- #FL) slam...
1,766192485,False,finalized,1,8/4/15 21:20,national,1.0,partisan,1.0,attack,...,NaN,NaN,NaN,M000355,"<blockquote class=""twitter-tweet"" width=""450"">...",3.11208E+17,From: Mitch McConnell (Senator from Kentucky),NaN,twitter,VIDEO - #Obamacare: Full of Higher Costs and ...
2,766192486,False,finalized,1,8/4/15 21:14,national,1.0,neutral,1.0,support,...,NaN,NaN,NaN,S001180,"<blockquote class=""twitter-tweet"" width=""450"">...",3.39069E+17,From: Kurt Schrader (Representative from Oregon),NaN,twitter,Please join me today in remembering our fallen...
3,766192487,False,finalized,1,8/4/15 21:08,national,1.0,neutral,1.0,policy,...,NaN,NaN,NaN,C000880,"<blockquote class=""twitter-tweet"" width=""450"">...",2.98528E+17,From: Michael Crapo (Senator from Idaho),NaN,twitter,RT @SenatorLeahy: 1st step toward Senate debat...
4,766192488,False,finalized,1,8/4/15 21:26,national,1.0,partisan,1.0,policy,...,NaN,NaN,NaN,U000038,"<blockquote class=""twitter-tweet"" width=""450"">...",4.07643E+17,From: Mark Udall (Senator from Colorado),NaN,twitter,.@amazon delivery #drones show need to update ...


In [77]:
matrix_description = df[['text', 'audience', 'bias', 'message']].values
for i in range(matrix_description.shape[0]):
    try:
        matrix_description[i, 0] = clearstring(matrix_description[i, 0])
    except:
        matrix_description[i, 0] = ''

In [79]:
unique_audience = np.unique(matrix_description[:, 1].astype('str')).tolist()
unique_bias = np.unique(matrix_description[:, 2].astype('str')).tolist()
unique_message = np.unique(matrix_description[:, 3].astype('str')).tolist()
print(unique_audience)
print(unique_bias)
print(unique_message)

['constituency', 'national']
['neutral', 'partisan']
['attack', 'constituency', 'information', 'media', 'mobilization', 'other', 'personal', 'policy', 'support']


In [83]:
for i in unique_audience:
    lists = matrix_description[np.where(matrix_description[:, 1] == i)[0], 0].tolist()
    lists = list(filter(None, lists))
    with open(i, 'w') as fopen:
        for k in range(len(lists) - 1):
            fopen.write(lists[k] + '\n')
        fopen.write(lists[-1])
    lists_name_entity = []
    for k in range(len(lists)):
        tokens = nlp(lists[k])
        lists[k] = ' '.join([str(token) for token in tokens if token.pos_ == 'VERB' or token.pos_ == 'NOUN'])
        lists_name_entity += [str(token) for token in tokens if token.ent_type_ == 'ORG' or token.ent_type_ == 'PERSON' or token.ent_type_ == 'GPE']
    with open(i + '_pos', 'w') as fopen:
        for k in range(len(lists) - 1):
            fopen.write(lists[k] + '\n')
        fopen.write(lists[-1])
    lists = list(set((' '.join(lists)).split()))
    with open(i + '_token', 'w') as fopen:
        for k in range(len(lists) - 1):
            fopen.write(lists[k] + '\n')
        fopen.write(lists[-1])
    lists_name_entity = list(set(lists_name_entity))
    with open(i + '_entity', 'w') as fopen:
        for k in range(len(lists_name_entity) - 1):
            fopen.write(lists_name_entity[k] + '\n')
        fopen.write(lists_name_entity[-1])

In [84]:
words = []
for i in unique_audience:
    direct = '/home/huseinzol05/Desktop/tree/audience/data/' + i + '/'
    with open(direct + os.listdir(direct)[0] , 'r') as fopen:
        words += fopen.read().split('\n')
        
vocab = list(set((' '.join(words)).split()))
unique_vocab = list(set(vocab))
with open('vocab_audience.p', 'wb') as fopen:
    pickle.dump(unique_vocab, fopen)
_, dictionary, reverse_dictionary = build_dictionary(vocab, len(unique_vocab))
with open('dictionary_audience.p', 'wb') as fopen:
    pickle.dump(dictionary, fopen)
with open('dict_reverse_audience.p', 'wb') as fopen:
    pickle.dump(reverse_dictionary, fopen)

In [85]:
for i in unique_bias:
    lists = matrix_description[np.where(matrix_description[:, 2] == i)[0], 0].tolist()
    lists = list(filter(None, lists))
    with open(i, 'w') as fopen:
        for k in range(len(lists) - 1):
            fopen.write(lists[k] + '\n')
        fopen.write(lists[-1])
    lists_name_entity = []
    for k in range(len(lists)):
        tokens = nlp(lists[k])
        lists[k] = ' '.join([str(token) for token in tokens if token.pos_ == 'VERB' or token.pos_ == 'NOUN'])
        lists_name_entity += [str(token) for token in tokens if token.ent_type_ == 'ORG' or token.ent_type_ == 'PERSON' or token.ent_type_ == 'GPE']
    with open(i + '_pos', 'w') as fopen:
        for k in range(len(lists) - 1):
            fopen.write(lists[k] + '\n')
        fopen.write(lists[-1])
    lists = list(set((' '.join(lists)).split()))
    with open(i + '_token', 'w') as fopen:
        for k in range(len(lists) - 1):
            fopen.write(lists[k] + '\n')
        fopen.write(lists[-1])
    lists_name_entity = list(set(lists_name_entity))
    with open(i + '_entity', 'w') as fopen:
        for k in range(len(lists_name_entity) - 1):
            fopen.write(lists_name_entity[k] + '\n')
        fopen.write(lists_name_entity[-1])

In [89]:
words = []
for i in unique_bias:
    direct = '/home/huseinzol05/Desktop/tree/bias/data/' + i + '/'
    with open(direct + os.listdir(direct)[0] , 'r') as fopen:
        words += fopen.read().split('\n')
        
vocab = list(set((' '.join(words)).split()))
unique_vocab = list(set(vocab))
with open('vocab_bias.p', 'wb') as fopen:
    pickle.dump(unique_vocab, fopen)
_, dictionary, reverse_dictionary = build_dictionary(vocab, len(unique_vocab))
with open('dictionary_bias.p', 'wb') as fopen:
    pickle.dump(dictionary, fopen)
with open('dict_reverse_bias.p', 'wb') as fopen:
    pickle.dump(reverse_dictionary, fopen)

In [93]:
for i in unique_message:
    lists = matrix_description[np.where(matrix_description[:, 3] == i)[0], 0].tolist()
    lists = list(filter(None, lists))
    with open(i, 'w') as fopen:
        for k in range(len(lists) - 1):
            fopen.write(lists[k] + '\n')
        fopen.write(lists[-1])
    lists_name_entity = []
    for k in range(len(lists)):
        tokens = nlp(lists[k])
        lists[k] = ' '.join([str(token) for token in tokens if token.pos_ == 'VERB' or token.pos_ == 'NOUN'])
        lists_name_entity += [str(token) for token in tokens if token.ent_type_ == 'ORG' or token.ent_type_ == 'PERSON' or token.ent_type_ == 'GPE']
    with open(i + '_pos', 'w') as fopen:
        for k in range(len(lists) - 1):
            fopen.write(lists[k] + '\n')
        fopen.write(lists[-1])
    lists = list(set((' '.join(lists)).split()))
    with open(i + '_token', 'w') as fopen:
        for k in range(len(lists) - 1):
            fopen.write(lists[k] + '\n')
        fopen.write(lists[-1])
    lists_name_entity = list(set(lists_name_entity))
    with open(i + '_entity', 'w') as fopen:
        for k in range(len(lists_name_entity) - 1):
            fopen.write(lists_name_entity[k] + '\n')
        fopen.write(lists_name_entity[-1])

In [95]:
words = []
for i in unique_message:
    direct = '/home/huseinzol05/Desktop/tree/message/data/' + i + '/'
    with open(direct + os.listdir(direct)[0] , 'r') as fopen:
        words += fopen.read().split('\n')
        
vocab = list(set((' '.join(words)).split()))
unique_vocab = list(set(vocab))
with open('vocab_message.p', 'wb') as fopen:
    pickle.dump(unique_vocab, fopen)
_, dictionary, reverse_dictionary = build_dictionary(vocab, len(unique_vocab))
with open('dictionary_message.p', 'wb') as fopen:
    pickle.dump(dictionary, fopen)
with open('dict_reverse_message.p', 'wb') as fopen:
    pickle.dump(reverse_dictionary, fopen)

In [96]:
list_sentiment = os.listdir('/home/huseinzol05/Desktop/tree/sentiment/data')
print(list_sentiment)

['negative', 'positive']


In [97]:
for i in list_sentiment:
    direct = '/home/huseinzol05/Desktop/tree/sentiment/data/' + i + '/'
    with open(direct + os.listdir(direct)[0] , 'r') as fopen:
        lists = fopen.read().split('\n')
    lists = list(filter(None, lists))
    with open(i, 'w') as fopen:
        for k in range(len(lists) - 1):
            lists[k] = clearstring(lists[k])
            fopen.write(lists[k] + '\n')
        lists[-1] = clearstring(lists[-1])
        fopen.write(lists[-1])
    lists_name_entity = []
    for k in range(len(lists)):
        tokens = nlp(lists[k])
        lists[k] = ' '.join([str(token) for token in tokens if token.pos_ == 'VERB' or token.pos_ == 'NOUN'])
        lists_name_entity += [str(token) for token in tokens if token.ent_type_ == 'ORG' or token.ent_type_ == 'PERSON' or token.ent_type_ == 'GPE']
    with open(i + '_pos', 'w') as fopen:
        for k in range(len(lists) - 1):
            fopen.write(lists[k] + '\n')
        fopen.write(lists[-1])
    lists = list(set((' '.join(lists)).split()))
    with open(i + '_token', 'w') as fopen:
        for k in range(len(lists) - 1):
            fopen.write(lists[k] + '\n')
        fopen.write(lists[-1])
    lists_name_entity = list(set(lists_name_entity))
    with open(i + '_entity', 'w') as fopen:
        for k in range(len(lists_name_entity) - 1):
            fopen.write(lists_name_entity[k] + '\n')
        fopen.write(lists_name_entity[-1])

In [98]:
words = []
for i in list_sentiment:
    direct = '/home/huseinzol05/Desktop/tree/sentiment/data/' + i + '/'
    with open(direct + os.listdir(direct)[0] , 'r') as fopen:
        words += fopen.read().split('\n')
        
vocab = list(set((' '.join(words)).split()))
unique_vocab = list(set(vocab))
with open('vocab_sentiment.p', 'wb') as fopen:
    pickle.dump(unique_vocab, fopen)
_, dictionary, reverse_dictionary = build_dictionary(vocab, len(unique_vocab))
with open('dictionary_sentiment.p', 'wb') as fopen:
    pickle.dump(dictionary, fopen)
with open('dict_reverse_sentiment.p', 'wb') as fopen:
    pickle.dump(reverse_dictionary, fopen)